<a href="https://colab.research.google.com/github/NewWinHilton/lastFirst/blob/master/darkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre-req

In [2]:
!git remote add origin https://github.com/NewWinHilton/firstLast.git
!git branch -M master
!git push -u origin master

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
!pip install gym
!pip install stable-baselines[mpi]
!brew install cmake openmpi
!pip install tensorflow==1.15


**1. Imports**

**2. Create Environment**

**3. Train Environment**

**4. Test Environment**

**5. Plot Results**

**6. Analysis Using quantstats**

In [ ]:
import gym
import json
import datetime as dt

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

from env.StockTradingEnv import StockTradingEnv

import pandas as pd

df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')

# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: StockTradingEnv(df)])

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=50000)

obs = env.reset()
for i in range(500000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float16
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/content/env/StockTradingEnv.py:79: RuntimeWarning: invalid value encountered in double_scalars
  prev_cost + additional_cost) / (self.shares_held + shares_bought)


Streaming output truncated to the last 5000 lines.
Profit: 0.0
Step: 2644
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 2645
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 2646
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 2647
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 2648
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 2649
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 

Snippets

In [ ]:


import numpy as np
import datetime as dt
import cbpro
import time
################################################################################
                             ## API STUFF ###

# CB Pro granted api credentials as strings
apiKey = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
apiSecret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
passphrase = "XXXXXXXXXXX"


auth_client = cbpro.AuthenticatedClient(apiKey,apiSecret,passphrase)

################################################################################
                        ### Investment Details ###

# Amount to initially invest
initInvestment = 20.00

# Amount that will be used for purchase starts at the initial amount
funding = initInvestment

# Currency to trade, for reference:
# 'BCH-USD' = Bitcoin Cash, 'BTC-USD' = Bitcoin, 'ETH-USD' = Ether
currency = 'BCH-USD'

# Will return the ID of your specific currency account
def getSpecificAccount(cur):
    x = auth_client.get_accounts()
    for account in x:
        if account['currency'] == cur:
            return account['id']

# Get the currency's specific ID
specificID = getSpecificAccount(currency[:3])

# Granularity (in seconds). So 300 = data from every 5 min
period = 300

# We will keep track of how many iterations our bot has done
iteration = 1

# Start off by looking to buy
buy = True

################################################################################
                ### Begin Loop and get Historic Data ###

while True:

    try:
        historicData = auth_client.get_product_historic_rates(currency, granularity=Period)

        # Make an array of the historic price data from the matrix
        price = np.squeeze(np.asarray(np.matrix(historicData)[:,4]))

        # Wait for 1 second, to avoid API limit
        time.sleep(1)

        # Get latest data and show to the user for reference
        newData = auth_client.get_product_ticker(product_id=currency)
        print(newData)
        currentPrice=newData['price']

    except:
        # In case something went wrong with cbpro
        print("Error Encountered")

################################################################################
                            ### Strategy ###

    # Calculate the rate of change 11 and 14 units back, then sum them
    ROC11 = np.zeros(13)
    ROC14 = np.zeros(13)
    ROCSUM = np.zeros(13)

    for ii in range(0,13):
        ROC11[ii] = (100*(price[ii]-price[ii+11]) / float(price[ii+11]))
        ROC14[ii] = (100*(price[ii]price[ii+14]) / float(price[ii+14]))
        ROCSUM[ii] = ( ROC11[ii] + ROC14[ii] )

    # Calculate the past 4 Coppock values with Weighted Moving Average
    for ll in range(0,4):
        coppock[ll] = (((1*ROCSUM[ll+9]) + (2*ROCSUM[ll+8]) + (3*ROCSUM[ll+7]) \
        + (4*ROCSUM[ll+6]) + (5*ROCSUM[ll+5]) + (6*ROCSUM[ll+4]) \
        + (7*ROCSUM[ll+3]) + (8*ROCSUM[ll+2]) + (9*ROCSUM[ll+1]) \
        + (10*ROCSUM[ll])) / float(55))

    # Calculate the past 3 derivatives of the Coppock Curve
    coppockD1 = np.zeros(3)
    for mm in range(3):
        coppockD1[mm] = coppock[mm] - coppock[mm+1]

################################################################################
                            ### Funds to Use ###

    # The maximum amount of Cryptocurrency that can be purchased with your funds
    possiblePurchase = (float(funding)) / float(currentPrice)

    # The amount of currency owned
    owned = float(auth_client.get_account(specificID)['available'])

    # The value of the cryptourrency in USD
    possibleIncome = float(currentPrice) * owned

################################################################################
                             ###Decision Making###

    # Buy Conditions: latest derivative is + and previous is -
    if buy == True and (coppockD1[0]/abs(coppockD1[0])) == 1.0 and (coppockD1[1]/abs(coppockD1[1])) == -1.0:

        # Place the order
        auth_client.place_market_order(product_id=currency, side='buy', funds=str(funding))

        # Print message in the terminal for reference
        message = "Buying Approximately " + str(possiblePurchase) + " " + \
        currency + "  Now @ " + str(currentPrice) + "/Coin. TOTAL = " + str(funding)
        print(message)

        # Update funding level and Buy variable
        funding = 0
        buy = False

    # Sell Conditions: latest derivative is - and previous is +
    if buy == False and (coppockD1[0]/abs(coppockD1[0])) == -1.0 and (coppockD1[1]/abs(coppockD1[1])) == 1.0:

        # Place the order
        auth_client.place_market_order(product_id=currency,side='sell',size=str(owned))

        # Print message in the terminal for reference
        message = "Selling " + str(owned) + " " + currency + "Now @ " + \
        str(currentPrice) + "/Coin. TOTAL = " + str(possibleIncome)
        print(message)

        # Update funding level and Buy variable
        funding = int(possibleIncome)
        buy = True

    # Stop loss: sell everything and stop trading if your value is less than 80% of initial investment
    if (possibleIncome+funding) <= 0.8 * initInvestment:

        # If there is any of the crypto owned, sell it all
        if owned > 0.0:
            auth_client.place_market_order(product_id = currency, side='sell', size = str(owned))
            print("STOP LOSS SOLD ALL")

        # Will break out of the while loop and the program will end
        break

    # Printing here to make the details easier to read in the terminal
    print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print("iteration number", iteration)

    # Print the details for reference
    print("Current Price: ", currentPrice)
    print("Your Funds = ", funding)
    print("You Own ", owned, "BCH")

    # Wait for 5 minutes before repeating
    time.sleep(300)
    iteration += 1

Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over time: 0.0
Score over ti

In [ ]:
import tensorflow as tf
print("tf", tf.__version__)

tf 2.3.0
